## A Demonstration of the Elliptic Curve Cryptosystem (ECC)
The elliptic curve cryptosystem demonstrated below combines [Diffie&ndash;Hellman](https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange) style symmetric key agreement with a digital signature to provide for the _confidentiality_, _integrity_ and _authenticy_ of a message exchanged between two parties (Alice and Bob) over an insecure channel. A third party (Eve) listening on this channel has no practical way of learning the content of the message, nor any way of tampering with it without the receiving party knowing the message has been corrupted.

The key&ndash;agreement component of this cryptosystem provides for message confidentiality, and is known by the initials _ECDH_, which is short for [_Elliptic&ndash;Curve Diffie&ndash;Hellman_](https://en.wikipedia.org/wiki/Elliptic-curve_Diffie%E2%80%93Hellman). The digital signature component of this cryptosystem provides for message integrity and authenticity, and is known by the initials _ECDSA_, for [_Elliptic Curve Digital Signature Algorithm_](https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm).

Elliptic curve cryptography represents a novel approach to the [discrete logarithm problem](https://en.wikipedia.org/wiki/Discrete_logarithm) (DLP) introduced by Diffie and Hellman in their 1976 paper [_New Directions in Cryptography_](https://ee.stanford.edu/~hellman/publications/24.pdf). The Diffie&ndash;Hellman protocol remains to this day the basis for key agreement over insecure channels. Elliptic&ndash;curve DLP has the advantage of requiring much shorter key lengths than traditional DH, however, to achieve equivalent security levels. This makes ECC the only practical choice in size&ndash; and/or power&ndash;constrained applications, such as contactless smartcards and smart phones. Whereas an ECC key of 256 bits in length provides 128&ndash;bit security (today considered the minimum acceptible security level), DH and [RSA](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) keys must be at least 3,072 bits in length to provide the same level of security.

### A Caveat
Note that the procedure as demonstrated provides no guarantee as to the authenticity of either party; that is, that the participants Alice or Bob (or both) are who they claim to be (see [_man&ndash;in&ndash;the&ndash;middle_](https://en.wikipedia.org/wiki/Man-in-the-middle_attack) attack for elaboration). Nor does it guarantee that the message encrypted in the procedure cannot be decrypted by an adversary if either private key used in the procedure is compromised at a future date (see [_forward secrecy_](https://en.wikipedia.org/wiki/Forward_secrecy) for elaboration). Protections against either vulnerability are not precluded by this implementation, but they are omitted here to minimize conceptual clutter.

### The Procedure
The following procedure, which utilizes the services of this repository's [ec](https://github.com/dchampion/crypto/tree/master/src/core/ec.py) module, illustrates a session in which Alice signs and encrypts a private message, and then transmits this message, together with the signature, to Bob over an insecure channel. Meanwhile, an adversary (Eve) intecepts the message&ndash;signature pair transmitted on the insecure channel.

It is assumed that Alice, Bob and Eve all possess their own copy of the [ec](https://github.com/dchampion/crypto/tree/master/src/core/ec.py) module; but that only Alice and Bob possess the signing and encryption keys generated by this module to provide the aforementioned protections.

Alice and Bob start by importing the modules [curves](https://github.com/dchampion/crypto/tree/master/src/core/curves.py) and [ec](https://github.com/dchampion/crypto/tree/master/src/core/ec.py), and a module [sym](https://github.com/dchampion/crypto/tree/master/src/tests/core/sym.py) that implements a symmetric cipher they have agreed in advance will be used to encrypt and decrypt messages (the [sym](https://github.com/dchampion/crypto/tree/master/src/tests/core/sym.py) cipher used here is for illustrative purposes only, and in a real&ndash;world setting would be replaced with a cryptographically&ndash;strong symmetric cipher such as _3DES_ or _AES_).

In [1]:
import os
os.chdir("../src")

# Alice and Bob (and Eve?) import the curves, ec and sym modules required for the session
from core import curves
from core import ec
from tests.core import sym

Alice generates a keypair, and stores it in `key_a`. This pair consists of both her private and public keys (the suffix `_a` denotes that this keypair belongs to Alice). In the second line of code, Alice retrieves her public key from `key_a` and stores it in `pub_key_a`. Alice transmits `pub_key_a` to Bob, but she must _not_ transmit `key_a` to Bob because it contains her private key.

In [2]:
# Alice computes a public-private key pair, and transmits her public key (pub_key_a) to Bob
key_a = ec.make_key()
pub_key_a = key_a.public_key()

Now it's Bob's turn to generate a keypair (suffixed with `_b` this time to denote that it belongs to Bob). Like Alice, Bob retrieves his public key from `key_b` and stores it in `pub_key_b`. He transmits `pub_key_b` to Alice.

In [3]:
# Bob computes his public-private key pair, and tranmits his public key (pub_key_b) to Alice
key_b = ec.make_key()
pub_key_b = key_b.public_key()

Next, using the public key she received from Bob, Alice computes a session key and stores it in `ses_key_a`.

In [4]:
# Alice computes a session key
ses_key_a = key_a.make_session_key(pub_key_b)

If you are familiar with traditional Diffie&ndash;Hellman, you might recognize a pattern here. The essential property of DH is expressed in the signature of the method `key_a.make_session_key()`; namely, a secret session key is computed using one party's private key in combination with the other party's public key (in the present case these are Alice's private key, hidden inside `key_a`, and Bob's public key `pub_key_b`).

Now Bob computes a session key in the same way Alice did, and stores it in `ses_key_b`. Bob supplies Alice's public key to `key_b.make_session_key()`.

In [5]:
# Bob computes a session key
ses_key_b = key_b.make_session_key(pub_key_a)

If the essential property of DH holds, Alice's and Bob's session keys (`ses_key_a` and `ses_key_b`) should be identical. Further, Eve, having only observed Alice's and Bob's public keys, cannot feasibly derive the session key with just that knowledge.

Here we see that `ses_key_a` is in fact equivalent to `ses_key_b`.

In [6]:
ses_key_a == ses_key_b

True

Alice composes a message (`mA`) for Bob.

In [7]:
# Alice composes a message for Bob
mA = "Sign and encrypt me!"

Alice then signs the message and stores the signature in `S`. To sign the message, Alice calls the `key_a.sign()` method, passing it `mA`.

In [8]:
# Alice signs the message, and transmits the signature (S) to Bob
S = key_a.sign(mA)

Next, Alice encrypts the message, supplying the message and her session key to the `encrypt()` function of the symmetric cipher. She stores the ciphertext in `mAC`.

In [9]:
# Alice encrypts the message, and transmits its ciphertext (mAC) to Bob
mAC = sym.encrypt(ses_key_a, mA)

Finally, Alice transmits the ciphertext (`mAC`), and the signature (`S`), to Bob.

We see here that the ciphertext is a random string of bytes that bears no resemblance to the plaintext, rendering it indecipherable to Eve.

In [10]:
mAC

b'&\x80\xae,\x07[~\xe9\xd0\xd5\xd4whGU\xe5\xdf\x8cr)\xfdE^\x1fG\xe8\xfb\xc2\xb6=k\xd9'

Bob, having received the ciphertext from Alice, decrypts it using the `decrypt()` function of the symmetric cipher. He stores the result in `mB`.

In [11]:
# Bob decrypts the ciphertext (mAC) of Alice's message
mB = sym.decrypt(ses_key_b, mAC)

Here we see that the deciphered plaintext (`mB`) matches the original plaintext (`mA`).

In [12]:
mB

'Sign and encrypt me!'

Thus far Alice has transmitted her message _confidentially_ to Bob, which is to say Alice and Bob can both be confident that it has not been read by Eve. What remains is for Bob to verify the _integrity_ and _authenticity_ of the message. These properties ensure that Eve has not tampered with or altered the message in any way.

Bob tests for these assurances by calling the `key_b.verify()` method, supplying Alice's public key, the plaintext of the message that he recovered in the previous decryption step, and the signature.

In [13]:
# Bob verifies the integrity and authenticity of Alice's message
key_b.verify(pub_key_a, mB, S)

True

If `key_b.verify()` returns `True`, Bob can be satisfied that the additional conditions of integrity and authenticity have been met, and the session is complete. If the method returns `False`, Bob must reject the message and assume the system is being attacked.